<a href="https://www.kaggle.com/code/achellesheel/titanic-pytorch-chatur?scriptVersionId=149799122" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Load the Titanic dataset
train_datax = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
passenger_id = test_data['PassengerId']
# Preprocess the data
def preprocess_data(data):
    # Select relevant features
    data = data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]  # Remove 'Survived' from the features

    # Impute missing values
    imputer = SimpleImputer(strategy='mean')  # Use mean imputation for missing values
    # Impute missing values for the 'Age' column
    data.loc[:, 'Age'] = imputer.fit_transform(data[['Age']])

    
    # Convert categorical variables to one-hot encoding
    data = pd.get_dummies(data, columns=['Sex', 'Embarked'], drop_first=True)

    return data

train_data = preprocess_data(train_datax)
test_data = preprocess_data(test_data)

# Handle missing values in the test data (if any)
test_data['Fare'].fillna(test_data['Fare'].mean(), inplace=True)

# Split the data into features and target variable
X = train_data  # Features
y = train_datax['Survived']  # Target variable

# Split the training data into a training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# Standardize the features
scaler = StandardScaler()
# Fit the scaler with feature names
scaler.fit(X_train)

# Transform the training data
X_train = scaler.transform(X_train)

# Transform the validation data
X_val = scaler.transform(X_val)

# Transform the test data
test_data = scaler.transform(test_data)


# Convert data to NumPy arrays
X_train = X_train.astype(np.float32)
y_train = y_train.values.astype(np.float32)
X_val = X_val.astype(np.float32)
y_val = y_val.values.astype(np.float32)

# Create PyTorch data loaders
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)
X_val = torch.tensor(X_val)
y_val = torch.tensor(y_val)

# Create PyTorch data loaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define the model
class TitanicModel(nn.Module):
    def __init__(self, input_size):
        super(TitanicModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

input_size = X_train.shape[1]
model = TitanicModel(input_size)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        optimizer.step()

# Evaluate the model on the validation set
model.eval()
with torch.no_grad():
    val_predictions = model(X_val).numpy()
val_predictions = (val_predictions > 0.5).astype(int)

# Make predictions on the test set
X_test = torch.tensor(test_data, dtype=torch.float32)
model.eval()
with torch.no_grad():
    test_predictions = model(X_test).numpy()
test_predictions = (test_predictions > 0.5).astype(int)



In [3]:
from sklearn.metrics import accuracy_score

# Calculate accuracy on the validation set
val_accuracy = accuracy_score(y_val, val_predictions)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")



Validation Accuracy: 81.56%


In [4]:
len(y_val)

179

In [5]:
len(test_predictions)

418

In [6]:
import pandas as pd

# Flatten passenger_id and test_predictions to ensure they are 1-dimensional
passenger_id = passenger_id.ravel()
test_predictions = test_predictions.ravel()

# Create the DataFrame
submission_df = pd.DataFrame({'PassengerId': passenger_id, 'Survived': test_predictions})

# Save the DataFrame as a CSV file
submission_df.to_csv('submission.csv', index=False)
